# Modèle SVM pour la reconnaissance de chiffres manuscrits

Ce notebook implémente un pipeline pour entraîner un modèle SVM sur le dataset MNIST et effectuer des prédictions.

In [1]:
# Importer les bibliothèques nécessaires
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pickle
import numpy as np

## Étape 1 : Télécharger et prétraiter les données MNIST

In [2]:
def load_and_preprocess_data():
    """
    Charger les données MNIST depuis OpenML, les binariser et les séparer en ensembles d'entraînement et de test.
    """
    # Télécharger le dataset MNIST
    mnist = fetch_openml('mnist_784', version=1)
    X, y = mnist.data, mnist.target.astype(int)  # X : pixels, y : étiquettes

    # Binariser les pixels (valeurs supérieures à 128 deviennent 1, sinon 0)
    X = (X > 128).astype(np.float32)

    # Diviser les données en ensembles d'entraînement (80%) et de validation (20%)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    return X_train, X_val, y_train, y_val

## Étape 2 : Entraîner le modèle

In [3]:
def train_svm(X_train, y_train):
    """
    Entraîner un modèle SVM sur les données binarisées.
    """
    svm_model = SVC(kernel='rbf', gamma=0.001, C=10)
    svm_model.fit(X_train, y_train)
    return svm_model

## Étape 3 : Évaluer le modèle

In [4]:
def evaluate_model(model, X_val, y_val):
    """
    Évaluer le modèle sur l'ensemble de validation.
    """
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    print("Rapport de classification :\n", classification_report(y_val, y_pred))
    print(f"Précision du modèle : {accuracy:.2f}")

## Étape 4 : Sauvegarder le modèle

In [ ]:
def save_model(model, filename='modelSVM.sav'):
    """
    Sauvegarder le modèle entraîné dans un fichier.
    """
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Modèle sauvegardé dans le fichier : {filename}")

## Étape 5 : Charger le modèle et prédire

In [6]:
def load_model(filename='finalized_model.sav'):
    """
    Charger le modèle sauvegardé pour une utilisation future.
    """
    with open(filename, 'rb') as f:
        return pickle.load(f)

def predict_digit(image_vector, model):
    """
    Prédire un chiffre manuscrit donné un vecteur de 784 valeurs (image aplatie 28x28).
    """
    prediction = model.predict([image_vector])
    return prediction[0]

## Étape 6 : Pipeline principal

In [11]:
filename = 'finalized_model.sav'
model = load_model(filename)

#recupérons les coefficients du svm à noyau RBF
#vecteurs de supports
vecteurs_supports = model.support_vectors_
#coefficients de chaque classe
coefficients = model.dual_coef_
#le bias
biais = model.intercept_

print("vecteurs de support:\n",vecteurs_supports)
print("taille")
print(vecteurs_supports.shape)

print("coefficients de chaque classe")
print("coefficients:\n",coefficients.shape)

print("taille du vecteur de biais")
print("bias:\n",biais.shape)
print

np.savetxt("vecteurs_support.txt",vecteurs_supports)
np.savetxt("coefficients.txt",coefficients)
np.savetxt("biais.txt",biais)



vecteurs de support:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
taille
(12941, 784)
coefficients de chaque classe
coefficients:
 (9, 12941)
taille du vecteur de biais
bias:
 (45,)


In [9]:
X_train, X_val, y_train, y_val = load_and_preprocess_data()
example_vector = X_val.iloc[0]  # Utiliser iloc pour accéder à la première ligne d'un DataFrame

predicted_digit = predict_digit(example_vector, model)
print(f"Exemple : Le chiffre prédit est : {predicted_digit}")
print(f"Étiquette réelle : {y_val.iloc[0]}")

Exemple : Le chiffre prédit est : 7
Étiquette réelle : 7


/Users/raniabouzroud/Desktop/analyse et visualisation de données/venv_global/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
